# Trabajo Práctico de Aprendizaje Automático Profundo

> **Integrantes**: Candela Spitale | Fernando Cardellino | Carina Giovine | Carlos Serra

## Inicialización de ambiente

In [1]:
import gzip
import bz2
import json
import mlflow
import pandas as pd
import random

import tempfile
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import IterableDataset, DataLoader
from tqdm.notebook import tqdm, trange

from sklearn.model_selection import train_test_split
from sklearn.metrics import balanced_accuracy_score

In [2]:
torch.__version__

'1.10.1+cu111'

In [3]:
torch.cuda.is_available()

True

In [4]:
# para usar GPU
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")

In [ ]:
# Descargamos los datasets
!curl -L https://cs.famaf.unc.edu.ar/\~ccardellino/resources/diplodatos/meli-challenge-2019.tar.bz2 -o ./data/meli-challenge-2019.tar.bz2
!tar jxvf ./data/meli-challenge-2019.tar.bz2 -C ./data/

In [ ]:
# Descargamos word embeddings
!curl -L https://cs.famaf.unc.edu.ar/~ccardellino/SBWCE/SBW-vectors-300-min5.txt.bz2 -o ./data/SBW-vectors-300-min5.txt.bz2

## 0. Análisis y Visualización

In [21]:
%%time
meli_df_train = pd.read_json(
    './data/meli-challenge-2019/spanish.train.jsonl.gz', lines=True)

meli_df_val = pd.read_json(
    './data/meli-challenge-2019/spanish.validation.jsonl.gz',lines=True)

meli_df_test = pd.read_json(
    './data/meli-challenge-2019/spanish.test.jsonl.gz',lines=True)

CPU times: user 2min 20s, sys: 36 s, total: 2min 56s
Wall time: 2min 55s


Vemos qué presenta cada conjunto

In [59]:
print("Conjunto de Entrenamiento\n")
meli_df_train.info()
meli_df_train.head(3)
print("\nDatos nulos\n")
meli_df_train.isnull().sum()

Conjunto de Entrenamiento

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4895280 entries, 0 to 4895279
Data columns (total 10 columns):
 #   Column           Dtype 
---  ------           ----- 
 0   language         object
 1   label_quality    object
 2   title            object
 3   category         object
 4   split            object
 5   tokenized_title  object
 6   data             object
 7   target           int64 
 8   n_labels         int64 
 9   size             int64 
dtypes: int64(3), object(7)
memory usage: 373.5+ MB

Datos nulos



language           0
label_quality      0
title              0
category           0
split              0
tokenized_title    0
data               0
target             0
n_labels           0
size               0
dtype: int64

In [60]:
print("Conjunto de Validación\n")
meli_df_val.info()
meli_df_val.head(3)
print("\nDatos nulos")
meli_df_val.isnull().sum()

Conjunto de Validación

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1223820 entries, 0 to 1223819
Data columns (total 10 columns):
 #   Column           Non-Null Count    Dtype 
---  ------           --------------    ----- 
 0   language         1223820 non-null  object
 1   label_quality    1223820 non-null  object
 2   title            1223820 non-null  object
 3   category         1223820 non-null  object
 4   split            1223820 non-null  object
 5   tokenized_title  1223820 non-null  object
 6   data             1223820 non-null  object
 7   target           1223820 non-null  int64 
 8   n_labels         1223820 non-null  int64 
 9   size             1223820 non-null  int64 
dtypes: int64(3), object(7)
memory usage: 93.4+ MB

Datos nulos


language           0
label_quality      0
title              0
category           0
split              0
tokenized_title    0
data               0
target             0
n_labels           0
size               0
dtype: int64

In [61]:
print("Conjunto de Test\n")
meli_df_test.info()
meli_df_test.head(3)
print("\nDatos nulos\n")
meli_df_test.isnull().sum()

Conjunto de Test

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63680 entries, 0 to 63679
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   language         63680 non-null  object
 1   label_quality    63680 non-null  object
 2   title            63680 non-null  object
 3   category         63680 non-null  object
 4   split            63680 non-null  object
 5   tokenized_title  63680 non-null  object
 6   data             63680 non-null  object
 7   target           63680 non-null  int64 
 8   n_labels         63680 non-null  int64 
 9   size             63680 non-null  int64 
dtypes: int64(3), object(7)
memory usage: 4.9+ MB

Datos nulos



language           0
label_quality      0
title              0
category           0
split              0
tokenized_title    0
data               0
target             0
n_labels           0
size               0
dtype: int64

Los 3 conjuntos no presentan datos nulos.

In [35]:
print("cantidad de categorías de train {}".format(len(meli_df_train.category.unique())))

cantidad de categorías de train 632


In [36]:
print("cantidad de categorías de train {}".format(len(meli_df_val.category.unique())))

cantidad de categorías de train 632


In [37]:
print("cantidad de categorías de train {}".format(len(meli_df_test.category.unique())))

cantidad de categorías de train 632


In [38]:
print("cantidad de títulos de train {}".format(len(meli_df_train.title.unique())))

cantidad de títulos de train 4895280


In [39]:
print("cantidad de títulos de val {}".format(len(meli_df_val.title.unique())))

cantidad de títulos de val 1223820


In [40]:
print("cantidad de títulos de test {}".format(len(meli_df_test.title.unique())))

cantidad de títulos de test 63680


In [9]:
#!curl -L https://cs.famaf.unc.edu.ar/~ccardellino/SBWCE/SBW-vectors-300-min5.txt.bz2 -o ./data/SBW-vectors-300-min5.txt.bz2

In [10]:
#embeddings = pd.read_csv('./data/SBW-vectors-300-min5.txt.bz2', nrows=20)

In [11]:
#embeddings

## 1. Hacer un preprocesamiento de los datos

Los conjuntos json de `spanish.{split}.json` ya están preprocesados y tokenizados, con las columnas `data` (referente a **title**) y `target` (referente a **category**). Esto se hizo en el [repositorio de la materia](https://github.com/DiploDatos/AprendizajeProfundo) en el archivo `experiment/preprocess_meli_data.ipynb`. Por lo cual, los utilizaremos directamente accediendo a estas columnas.


## 2. Tener un manejador del dataset

In [5]:
class MeliChallengeDataset(IterableDataset):
    def __init__(self,
                 dataset_path,
                 random_buffer_size=2048):
        assert random_buffer_size > 0
        self.dataset_path = dataset_path
        self.random_buffer_size = random_buffer_size

        with gzip.open(self.dataset_path, "rt") as dataset:
            item = json.loads(next(dataset).strip())
            self.n_labels = item["n_labels"]
            self.dataset_size = item["size"]

    def __len__(self):
        return self.dataset_size

    def __iter__(self):
        try:
            with gzip.open(self.dataset_path, "rt") as dataset:
                shuffle_buffer = []

                for line in dataset:
                    item = json.loads(line.strip())
                    item = {
                        "data": item["data"],
                        "target": item["target"]
                    }

                    if self.random_buffer_size == 1:
                        yield item
                    else:
                        shuffle_buffer.append(item)

                        if len(shuffle_buffer) == self.random_buffer_size:
                            random.shuffle(shuffle_buffer)
                            for item in shuffle_buffer:
                                yield item
                            shuffle_buffer = []

                if len(shuffle_buffer) > 0:
                    random.shuffle(shuffle_buffer)
                    for item in shuffle_buffer:
                        yield item
        except GeneratorExit:
            return

In [7]:
train_dataset = MeliChallengeDataset('./data/meli-challenge-2019/spanish.train.jsonl.gz')

val_dataset = MeliChallengeDataset('./data/meli-challenge-2019/spanish.validation.jsonl.gz')

test_dataset = MeliChallengeDataset('./data/meli-challenge-2019/spanish.test.jsonl.gz')

In [ ]:
class PadSequences:
    def __init__(self, pad_value=0, max_length=None, min_length=1):
        assert max_length is None or min_length <= max_length
        self.pad_value = pad_value
        self.max_length = max_length
        self.min_length = min_length

    def __call__(self, items):
        data, target = list(zip(*[(item["data"], item["target"]) for item in items]))
        seq_lengths = [len(d) for d in data]

        if self.max_length:
            max_length = self.max_length
            seq_lengths = [min(self.max_length, l) for l in seq_lengths]
        else:
            max_length = max(self.min_length, max(seq_lengths))

        data = [d[:l] + [self.pad_value] * (max_length - l)
                for d, l in zip(data, seq_lengths)]
            
        return {
            "data": torch.LongTensor(data),
            "target": torch.LongTensor(target)
        }

In [8]:
pad_sequences = PadSequences(
                    pad_value=0,
                    max_length=None,
                    min_length=1)

train_loader = DataLoader(train_dataset, batch_size=128,
                          collate_fn=pad_sequences, drop_last=False)

val_loader = DataLoader(val_dataset, batch_size=128,
                         collate_fn=pad_sequences, drop_last=False)

test_loader = DataLoader(test_dataset, batch_size=128, 
                         collate_fn=pad_sequences, drop_last=False)

## 3. Crear una clase para el modelo

In [9]:
class MLC_Classifier(nn.Module):
    def __init__(self, 
                 pretrained_embeddings_path, 
                 token_to_index,
                 vector_size,
                 freeze_embedings,
                 hidden1_size,
                 hidden2_size,
                 hidden3_size,
                 hidden4_size):

        super().__init__()
        with gzip.open(token_to_index, "rt") as fh:
              token_to_index = json.load(fh)
        embeddings_matrix = torch.randn(len(token_to_index), vector_size)
        embeddings_matrix[0] = torch.zeros(vector_size)
        
        with bz2.open(pretrained_embeddings_path, "rt") as fh:
            for line in fh:
                word, vector = line.strip().split(None, 1)
                if word in token_to_index:
                    embeddings_matrix[token_to_index[word]] =\
                        torch.FloatTensor([float(n) for n in vector.split()])
        
        self.embeddings = nn.Embedding.from_pretrained(embeddings_matrix,
                                                       freeze=freeze_embedings,
                                                       padding_idx=0)
        
        self.hidden1 = nn.Linear(vector_size, hidden1_size)
        self.hidden2 = nn.Linear(hidden1_size, hidden2_size)
        self.hidden3 = nn.Linear(hidden2_size, hidden3_size)
        self.hidden4 = nn.Linear(hidden3_size, hidden4_size)
        self.output = nn.Linear(hidden4_size, 632)
        self.vector_size = vector_size
    
    def forward(self, x):
        x = self.embeddings(x)
        x = torch.mean(x,dim=1) # estandarizamos el ancho de las matrices
        x = F.relu(self.hidden1(x)) # función de activación de la capa 1
        x = F.relu(self.hidden2(x))
        x = F.relu(self.hidden3(x))
        x = F.relu(self.hidden4(x))
        x = self.output(x)
        return x

## 4. Hacer logs de entrenamiento. Usar MLFlow.

In [10]:
def train_model(model, optimizer, loss_function, trainloader, epochs, use_tqdm=True):
    model.train()  # Tell the model to set itself to "train" mode.
    for epoch in range(epochs):  # loop over the dataset multiple times
        running_loss = []
        running_loss_displayed = 0.0
        pbar = tqdm(trainloader) if use_tqdm else trainloader
        for step, data in enumerate(pbar, 1):
            # get the inputs; data is a list of [inputs, labels]
            inputs = data["data"].to(device)
            labels = data["target"].to(device)

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = model(inputs)
            loss = loss_function(outputs, labels)
            loss.backward()
            optimizer.step()

            # print statistics
            running_loss.append(loss.item())
            running_loss_displayed += loss.item()
            if use_tqdm and step > 0 and step % 50 == 0: # print every 50 mini-batches
                pbar.set_description(f"[{epoch + 1}, \
                                    {step}] loss: {running_loss_displayed / step:.4g}")
    return running_loss


Para ver cómo predice el modelo en un dataset que no conoce (dataset de `test`) necesitamos usar una métrica de evaluación. Dado que estamos con un problema multiclase, en este caso vamos a usar el **balance accuracy score**. 

A fin de probar configuraciones, arquitecturas o hiperparámetros y encontrar los mejores para el conjunto de `test`, utilizaremos esta métrica en el conjunto de `entrenamiento` y `validación`.

In [11]:
def train_and_eval(model, optimizer, loss_function, trainloader, epochs,
                   valloader, use_tqdm=True):

    history = {'train_loss': [], 'val_loss': [], 'val_balanced_accuracy': []}
    for epoch in range(epochs):  # loop over the dataset multiple times
        print("Iniciando train con datos de entrenamiento")
        model.train()
        running_loss = 0.0
        pbar = tqdm(trainloader) if use_tqdm else trainloader
        for step, data in enumerate(pbar, 1):
            # get the inputs; data is a list of [inputs, labels]
            inputs = data["data"].to(device)
            labels = data["target"].to(device)
            # zero the parameter gradients
            optimizer.zero_grad()
            # forward + backward + optimize
            outputs = model(inputs.view(inputs.shape[0], -1))
            loss = loss_function(outputs, labels)
            loss.backward()
            optimizer.step()

            # print statistics
            running_loss += loss.item()
            if use_tqdm and step > 0 and step % 50 == 0:
                pbar.set_description(
                    f"[{epoch + 1}, {step}] loss: {running_loss / step:.4g}")

        history['train_loss'].append((epoch, running_loss / step))

        # At the end of the epoch, evaluate model on validation
        print("Iniciando eval con datos de validación")
        model.eval();  # Activate evaluation mode
        running_loss = 0.0
        y_true = []
        y_pred = []
        with torch.no_grad():
            pbar = tqdm(valloader) if use_tqdm else valloader
            for step, data in enumerate(pbar, 1):
                inputs = data["data"].to(device)
                labels = data["target"].to(device)
                outputs = model(inputs.view(inputs.shape[0], -1))
                running_loss += loss_function(outputs, labels).item()
                _, predicted = torch.max(outputs.data, 1)
                y_true.extend(labels.cpu().numpy())
                y_pred.extend(predicted.cpu().numpy())

        history['val_balanced_accuracy'].append(balanced_accuracy_score(y_true, y_pred))
        history['val_loss'].append((epoch, running_loss / step))
    
    return history

In [12]:
def test_model(model, dataloader, prefix='test_', use_tqdm=True):
    model.eval();  # Activate evaluation mode
    running_loss = []
    y_true = []
    y_pred = []
    with torch.no_grad():
        pbar = tqdm(dataloader) if use_tqdm else dataloader
        print("Iniciando experimento con conjunto de test")
        for data in pbar:
            inputs = data["data"].to(device)
            labels = data["target"].to(device)

            outputs = model(inputs.view(inputs.shape[0], -1))
            _, predicted = torch.max(outputs.data, 1)
            y_true.extend(labels.cpu().numpy())
            y_pred.extend(predicted.cpu().numpy())

    return {
        prefix + 'balanced_accuracy': balanced_accuracy_score(y_true, y_pred)
    }

In [13]:
def run_experiment(model, epochs, trainloader, valloader, testloader=None, 
                    optimizer_class=optim.SGD, lr=0.001, weight_decay=0.0,
                    use_tqdm=True):

    loss_function = nn.CrossEntropyLoss()
    optimizer = optimizer_class(model.parameters(), lr=lr,
                                weight_decay=weight_decay)
    history = train_and_eval(
        model, optimizer, loss_function, trainloader, epochs,
        valloader=valloader, use_tqdm=use_tqdm)
    
    if testloader:
        test_results = test_model(model, testloader, use_tqdm=use_tqdm)

    experiment = {
        'arquitecture': str(model), 'loss': str(loss_function),
        'epochs': epochs, 'lr': lr, 'optimizer': str(optimizer_class),
        'weight_decay': weight_decay
    }
    
    experiment.update(history)

    if testloader:
        experiment.update(test_results)
    
    return experiment



In [14]:
def run__mlflow_experiment(indice, epochs,
                           hidden1_size, hidden2_size, hidden3_size, hidden4_size,
                           optimizer_class,
                           lr,
                           run_name,
                           run_in_test=False):

    with mlflow.start_run(run_name=run_name):
        # como primer parámetro pasamos el archivo con los word embeddings descargados,
        # como segundo parámetro pasamos los títulos tokenizados y las categorías 
        # tokenizadas con sus respectivos índices, 
        # resultante de la tokenización hecha en la carpeta experiments de la materia
        model = MLC_Classifier("./data/SBW-vectors-300-min5.txt.bz2",
                               "./data/meli-challenge-2019/spanish_token_to_index.json.gz",
                               300, True, 
                               hidden1_size, hidden2_size, hidden3_size, hidden4_size)
        
        model.to(device)
        epochs = epochs
        
        mlflow.log_param("model_name", "MLC_Classifier")
        mlflow.log_param("freeze_embedding", False)
        mlflow.log_params({
            "embedding_size": 300,
            "hidden1_size": hidden1_size,
            "hidden2_size": hidden2_size,
            "hidden3_size": hidden3_size,
            "hidden4_size": hidden4_size,            
            "optimizer": optimizer_class,
            "lr": lr
        })
        print("Exploring ", optimizer_class, lr)
        
        if run_in_test:
            experiment = run_experiment(model, epochs, 
                                        train_loader, val_loader, 
                                        testloader=test_loader, 
                                        optimizer_class=optimizer_class, lr=lr)
        else:
            experiment = run_experiment(model, epochs, train_loader, val_loader, 
                                        optimizer_class=optimizer_class, lr=lr)
            
        for m in range(epochs):
            mlflow.log_metrics({
                'train_loss': experiment['train_loss'][m][1],
                'val_loss':experiment['val_loss'][m][1],
                'val_balanced_accuracy':experiment['val_balanced_accuracy'][m],    
            },m)
    
        if run_in_test:
            mlflow.log_metrics({
                'test_balanced_accuracy':experiment['test_balanced_accuracy']
            })

        print("Creando artefacto de MlFlow")
        with tempfile.TemporaryDirectory() as tmpdirname:
            targets = []
            predictions = []
            for batch in tqdm(val_loader):
                inputs = batch["data"].to(device)
                labels = batch["target"].to(device)
                output = model(inputs)
                targets.extend(labels.cpu().numpy())
                _, predicted = torch.max(output.data, 1)
                predictions.extend(predicted.cpu().numpy())

            pd.DataFrame({"prediction": predictions, "target": targets}).to_csv(
                f"{tmpdirname}/predictions_l1-{hidden1_size}"+
                f"size_l2-{hidden2_size}-sizel3-{hidden3_size}"+
                f"size_l4-{hidden4_size}size_iteracion{indice}.csv.gz", 
                    index=False
            )
            mlflow.log_artifact(
                f"{tmpdirname}/predictions_l1-{hidden1_size}"+
                f"size_l2-{hidden2_size}-sizel3-{hidden3_size}"+
                f"size_l4-{hidden4_size}size_iteracion{indice}.csv.gz"
            )
        
        return experiment

En todos los runs del experimento hacemos una red perceptrón multicapa de 4 capas ocultas con los siguientes tamaños:

* 1024 para la primera capa, considerando que el tamaño de input es 300, vamos aumentando las dimensiones
* 2048 para la segunda capa oculta
* 4096 para la tercera capa oculta
* 2048 para la última capa, que discrimina entre 632 categorías de salida (casi el doble que el tamaño de input) reduciendo a la mitad en relación a la 3er capa oculta y duplicando en relación a la 1ra capa.

Además, utilizamos `3` **épocas**.

Variamos únicamente el **optimizador** y la **taza de aprendizaje**.

In [16]:
epochs = 3
op_experiments = []
mlflow.set_experiment(experiment_name=f"experiment_w_{epochs}_epochs_l4")
indice = 0

for optimizer_class in [optim.Adam, optim.RMSprop]:
    for lr in [0.0001, 0.001]:

        run_name = (
            f"op_class:{optimizer_class}_lr:{lr}_torch"
        )
        experiment=run__mlflow_experiment(indice,
                                          epochs,
                                          1024,2048,4096,2048,
                                          optimizer_class,
                                          lr,
                                          run_name=run_name)
        op_experiments.append(experiment)
        indice+=1

2022/10/27 23:14:58 INFO mlflow.tracking.fluent: Experiment with name 'experiment_w_3_epochs_l4' does not exist. Creating a new experiment.


Exploring  <class 'torch.optim.adam.Adam'> 0.0001
Iniciando train con datos de entrenamiento


  0%|          | 0/38245 [00:00<?, ?it/s]

Iniciando eval con datos de validación


  0%|          | 0/9562 [00:00<?, ?it/s]

...

  0%|          | 0/38245 [00:00<?, ?it/s]

Iniciando eval con datos de validación


  0%|          | 0/9562 [00:00<?, ?it/s]

Creando artefacto de MlFlow


  0%|          | 0/9562 [00:00<?, ?it/s]

In [17]:
pd.DataFrame(op_experiments).to_csv(
    f"./data/op_experiments_w_{epochs}epochs_4l.csv.gz"
)

## 5. Hacer un gráfico de la función de loss a lo largo de las epochs

Usamos MLFlow y luego edición para identificar cada run, ya que en MLFlow no se lograban ver los hiperparámetros en el gráfico.

> Comparación de loss a través de las 3 épocas en conjunto de **entrenamiento** y **validación**

<img src='https://drive.google.com/uc?id=1z84GJLy4O_fcD8exVrw3eTEs228znyiP' name='train_loss'>

<img src='https://drive.google.com/uc?id=1l0yIOfcAWIz1G4JTLmtNYiwVN8PHvyd4' name='val_loss'>

## 6. Reportar performance en el conjunto de test con el mejor modelo entrenado.

In [33]:
epochs = 3
op_experiments = []
mlflow.set_experiment(experiment_name=f"test_experiment_w_{epochs}_epochs")
indice = 0

optimizer_class = optim.Adam
lr = 0.0001

run_name = f"test_op_class:{optimizer_class}_lr:{lr}_torch-mean"

experiment = run__mlflow_experiment(indice,
                                  epochs,
                                  1024,2048,4096,2048,
                                  optimizer_class,
                                  lr,
                                  run_name=run_name,
                                  run_in_test=True)
op_experiments.append(experiment)
indice+=1

pd.DataFrame(op_experiments).to_csv(
    f"./data/test_op_experiments_w_{epochs}epochs_4l.csv.gz")

Exploring  <class 'torch.optim.adam.Adam'> 0.0001
Iniciando train con datos de entrenamiento


  0%|          | 0/38245 [00:00<?, ?it/s]

Iniciando eval con datos de validación


  0%|          | 0/9562 [00:00<?, ?it/s]

Iniciando train con datos de entrenamiento


...

Iniciando eval con datos de validación


  0%|          | 0/9562 [00:00<?, ?it/s]

  0%|          | 0/498 [00:00<?, ?it/s]

Iniciando experimento con conjunto de test
Creando artefacto de MlFlow


  0%|          | 0/9562 [00:00<?, ?it/s]

> Performance del conjunto de test con el mejor modelo entrenado: Optim: Adam, LR: 0.0001

<img src='https://drive.google.com/uc?id=1W2siwQs6U-V1rYJez5bv5s7bs2j4_4DQ' name='val_loss' name='test_balanced_accuracy'>